# Обучение модели на архитектуре DistilBERT для предсказания услуг по страховым письмам

---



### Введение

В этом ноутбуке мы проведем тонкую настройку модели-трансформера для задачи **классификации текста с несколькими метками**.
Это одна из наиболее распространенных бизнес-задач, когда заданный фрагмент текста/предложение /документ необходимо классифицировать по одной или нескольким категориям из приведенного списка.

#### Разделы ноутбука

Ноутбук будет разделен на отдельные разделы, чтобы обеспечить организованное ознакомление с используемым процессом. Этот процесс может быть изменен для отдельных случаев использования. Разделы:

1. [Импорт библиотек Python и подготовка среды](#section01)
2. [Импорт и предварительная обработка данных](#section02)
3. [Подготовка набора данных и загрузчика данных](#section03)
4. [Создание нейронной сети для точной настройки](#section04)
5. [Точная настройка модели](#section05)
6. [Проверка работоспособности модели](#section06)
7. [Сохранение модели для будущего использования](#section07)

#### Техническая информация

Этот скрипт использует множество инструментов, разработанных другими командами. Подробная информация об используемых инструментах приведена ниже. Пожалуйста, убедитесь, что эти элементы присутствуют в вашей настройке для успешной реализации этого скрипта.

 - Данные:
	 - Мы используем данные, предоставленные "Медси", содержащие список электронных писем страховых компаний с размеченными услугами
	 - Данные разделены на 3 файла: для обучения (с размеченными кодами услуг), для валидации (с общей разметкой - есть услуги или нет) и справочник услуг с внутренними кодами
	 - Каждая строка обучающего датасета содержит:
		 - ID письма (letter_id)
		 - Текст письма (guarantee_letter_text)
		 - Коды услуг (service_ids_list)
		 - Признак наличия услуг (class)


Каждое письмо может быть одновременно отнесено к нескольким услугам. В этом случае в поле "Коды услуг" будет перечислено несколько кодов, например `[1, 2, 3]`

 - Используемая языковая модель:
	 - DistilBERT это модель-трансформер меньшего размера по сравнению с BERT или Roberta.  
	 - [Пример использования](https://medium.com/huggingface/distilbert-8cf3380435b5)
	 - [Научная статья](https://arxiv.org/pdf/1910.01108)
     - [Документация](https://huggingface.co/transformers/model_doc/distilbert.html)


 - Системные требования:
	 - Python 3.6 или выше
	 - Pytorch, Transformers и все стандартные библиотеки Python ML
	 - доступная GPU


 - Цель скрипта:
	 - Точно настроить DistilBERT чтобы иметь возможность относить электронные письма к перечню оказываемых услуг

---
***Примечание***
- *Следует отметить, что общие механизмы для задач с мультиклассом и несколькими метками схожи, за исключением нескольких отличий, а именно:*
	- *Функция потерь предназначена для оценки всей вероятности категорий по отдельности, а не по сравнению с другими категориями. Следовательно, при определении потерь используется `BCE`, а не `Cross Entropy`.*
	- *[Функция потерь](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.hamming_loss.html) и **Метрика Хэмминга** используются для прямого сравнения ожидаемого и прогнозируемого*
---

<a id='section01'></a>
### Импорт библиотек Python и подготовка среды

На этом шаге мы импортируем библиотеки и модули, необходимые для запуска нашего скрипта. Список библиотек:
* warnings
* Numpy
* Pandas
* tqdm
* scikit-learn metrics
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* DistilBERT Model and Tokenizer
* logging

После этого мы подготовим устройство к выполнению CUDA. Эта конфигурация необходима, если вы хотите использовать встроенный графический процессор.

In [1]:
# ! pip install transformers==3.0.2

In [2]:
# Импорт стандартных библиотек для ML
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel, AutoTokenizer, AutoModel
import logging
import ast
from bert_binary_classifier import BertBinaryClassifier
logging.basicConfig(level=logging.ERROR)

In [ ]:
binary_classifier = BertBinaryClassifier()

In [ ]:
print(binary_classifier.predict(
    """СОГАЗ 
 
Организовать приемы врача гинеколога, невролога, необходимые лабораторные и инструментальные исследования, 
манипуляции в рамках стандартной программы ДМС 
г. Москва, Ленинский проспект, д. 20, к.1 
 
Гарантируем оплату медицинских услуг по установленному страховому событию в соответствии 
с предусмотренной договором Программой и согласованным Прейскурантом услуг. Услуги, 
оказанные без медицинских показаний, а также отсутствующие в согласованном сторонами 
Прейскуранте, действующем на дату оказания услуги, оплате не подлежат."""
))

In [ ]:
print(binary_classifier.predict(
    """Ингосстрах 
 
F69.20.03.0.002 Компьютерная томография органов грудной 
полости (легкие, средостение) 
 
Оплата будет осуществляться в соответствии с условиями Договора и Программой, являющейся приложением к 
Договору. Оплата медицинских услуг, оказанных по данному гарантийному письму, может быть оспорена по 
результатам экспертизы счета и/или медицинской документации. 
Диагноз: - Интерстициальная болезнь легких неуточненная. J84.1 ? Эмфизема легких??? ДН-0-1 
Медси на Красной Пресне"""
))

In [69]:
# Проверка использования GPU

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [70]:
# Метод для расчета метрики Хэмминга
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/\
                    float( len(set_true.union(set_pred)) )
        acc_list.append(tmp_a)
    return np.mean(acc_list)

<a id='section02'></a>
### Импорт и предварительная обработка данных

Мы будем работать с данными и подготавливать их к точной настройке модели.
*Файл `train.xlsx` должен находиться вместе с ноутбуком*

* На первом шаге мы удаляем **id** письма из датасета.
* Создается новый датафрейм с колонкой **text**, в которую добавляется код услуги по справочнику.
* Значения всех кодов услуг преобразуются в массив.
* Массив кодов преобразуется в массив меток (0 или 1) для каждой услуги в колонке **labels**.

In [115]:
#data = pd.read_csv('Augm_train1.csv')
data = pd.read_excel('train.xlsx')

In [116]:
data.head()

,letter_id,guarantee_letter_text,service_ids_list,class
0,30624374,Ингосстрах \n \nF70.04.02.0.001 Магнитно-резон...,"[32, 18]",1
1,30637539,Согаз \n \nОрганизовать приемы врача (ревматол...,[75],0
2,30546372,Согаз \n \nОрганизовать приемы врача терапе...,[75],0
3,30718879,СОГАЗ \n \nОрганизовать приемы врача офталь...,[75],0
4,30597751,"АО ""АльфаСтрахование"" г.Москва благодарит з...","[34, 39]",1


In [117]:
data.drop(['letter_id', 'class'], inplace=True, axis=1)

In [118]:
# Общие параметры обучения

# Определим переменные, используемые при обучении
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 4
EPOCHS = 25
LEARNING_RATE = 1e-05
NUM_CLASSES = 76
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)
#tokenizer = AutoTokenizer.from_pretrained("Geotrend/distilbert-base-ru-cased")
#tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/ruBert-base", truncation=True)

In [101]:
def str_to_list(string):
  arr = ast.literal_eval(string)
  new_arr = []
  for i in range(NUM_CLASSES):
    if i in arr:
      new_arr.append(1)
    else:
      new_arr.append(0)

  return new_arr

In [100]:
def services_codes(string):
  arr = ast.literal_eval(string)
  result = ''
  for code in arr:
    result += ', '
    result += code

  return result

In [119]:
new_df = pd.DataFrame()
#new_df['text'] = data['guarantee_letter_text'] + data['ServiceCode'].apply(services_codes)
new_df['text'] = data['guarantee_letter_text']
#new_df['labels'] = data.iloc[:, 1:].values.tolist()
new_df['labels'] = data['service_ids_list'].apply(str_to_list)

In [120]:
new_df.head(3)

,text,labels
0,Ингосстрах \n \nF70.04.02.0.001 Магнитно-резон...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Согаз \n \nОрганизовать приемы врача (ревматол...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Согаз \n \nОрганизовать приемы врача терапе...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


<a id='section03'></a>
### Подготовка набора данных и загрузчика данных

Мы начнем с определения нескольких ключевых переменных, которые будут использоваться позже на этапе обучения/точной настройки.
Затем следует создание класса набора данных с несколькими метками - это определяет, как текст предварительно обрабатывается перед отправкой его в нейронную сеть. Мы также определим загрузчик данных (Dataloader), который будет передавать данные пакетами в нейронную сеть для соответствующего обучения и обработки.
Dataset и Dataloader - это конструкции библиотеки PyTorch для определения и управления предварительной обработкой данных и их передачей в нейронную сеть. Более подробно можно ознакомиться здесь [Документация PyTorch](https://pytorch.org/docs/stable/data.html)

#### *MultiLabelDataset* Dataset Class
- Этот класс принимает `tokenizer`, `dataframe` и `max_length` в качестве входных данных и генерирует токенизированные выходные данные и теги, которые используются моделью BERT для обучения.
- Мы используем токенизатор DistilBERT для токенизации данных в столбце `text` датафрейма.
- Токенизатор использует метод `encode_plus` для выполнения токенизации и генерации необходимых выходных данных: `ids`, `attention_mask`, `token_type_ids`

- Подробнее ознакомиться с токенизатором можно [по ссылке](https://huggingface.co/transformers/model_doc/distilbert.html#distilberttokenizer)
- `targets` это список услуг, обозначенных `0` или `1` в датафрейме.
- Класс *MultiLabelDataset* создает 2 датасета, для обучения и валидации.
- *Обучающий Датасет* используется для точной настройки модели: **80% от исходного датасета**
- *Валидационный Датасет* используется для оценки производительности модели. Модель не видела эти данные во время обучения.

#### Dataloader
- Dataloader используется для создания обучающего и валидационного загрузчика данных, который загружает данные в нейронную сеть определенным образом. Это необходимо, поскольку все данные из набора данных не могут быть загружены в память сразу, следовательно, необходимо контролировать объем данных, загружаемых в память и затем передаваемых в нейронную сеть.
- Этот контроль достигается с помощью таких параметров, как `batch_size` и `max_len`.
- Загрузчики обучающих и валидационных данных используются в обучающей и валидационной частях соответственно

In [121]:
class MultiLabelDataset(Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.labels
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [122]:
# Создание датасета и загрузчика данных

train_size = 0.8
train_data=new_df.sample(frac=train_size,random_state=200)
test_data=new_df.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = MultiLabelDataset(train_data, tokenizer, MAX_LEN)
testing_set = MultiLabelDataset(test_data, tokenizer, MAX_LEN)

FULL Dataset: (162, 2)
TRAIN Dataset: (130, 2)
TEST Dataset: (32, 2)


In [123]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

<a id='section04'></a>
### Создание нейронной сети для точной настройки

#### Нейронная сеть
 - Мы будем создавать нейронную сеть с помощью `DistilBERTClass`.
 - Эта сеть будет основана на модели `DistilBERT`.  За ней идут слои `Dropout` и `Linear`. Они добавлены с целью **Регуляризации** и **Классификации** соответственно.
 - В прямом цикле есть 2 выхода из слоя `DistilBERTClass`.
 - Второй выход `output_1` или по-другому `pooled output` передается в слой `Dropout`, а последующий выходной сигнал в слой `Linear`.
 - Обратите внимание, что количество измерений для слоя `Linear` равно **75**, потому что это общее количество услуг, по которым мы хотим классифицировать нашу модель.
 - Данные будут переданы в `DistilBERTClass`, как определено в наборе данных.
 - Выходные данные конечного слоя - это то, что будет использоваться для расчета потерь и определения точности прогнозирования моделей.
 - Мы создадим экземпляр сети под названием `model`. Этот экземпляр будет использоваться для обучения, а затем для сохранения окончательной обученной модели для будущего вывода.

#### Функция потерь и оптимизатор
 - Функция потерь определена далее как `loss_fn`.
 - Используемая функция потерь будет представлять собой комбинацию двоичной перекрестной энтропии, которая реализована следующим образом [BCELogits Loss](https://pytorch.org/docs/stable/nn.html#bcewithlogitsloss) в PyTorch
 - `Optimizer` определен ниже.
 - `Optimizer` используется для обновления весов нейронной сети с целью повышения ее производительности.

In [124]:
# Создаем модель, добавляя слои drop out и полносвязный поверх distil bert, чтобы получить финальные данные от модели.

class DistilBERTClass(torch.nn.Module):
    def __init__(self):
        super(DistilBERTClass, self).__init__()
        #self.l1 = DistilBertModel.from_pretrained("distilbert-base-uncased")
        self.l1 = AutoModel.from_pretrained("Geotrend/distilbert-base-ru-cased")
        #self.l1 = AutoModel.from_pretrained("sberbank-ai/ruBert-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        #self.classifier = torch.nn.Linear(768, 6)
        self.classifier = torch.nn.Linear(768, NUM_CLASSES)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

model = DistilBERTClass()
model.to(device)

DistilBERTClass(
  (l1): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(14283, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): Linear(in

In [125]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [126]:
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

<a id='section05'></a>
### Точная настройка модели

Здесь мы определяем обучающую функцию, которая обучает модель на обучающем наборе данных, созданном выше, указанное количество раз (эпох). Эпоха определяет, сколько раз полные данные будут переданы через сеть.

В этой функции происходят следующие события для точной настройки нейронной сети:
- Загрузчик данных передает данные в модель на основе размера пакета.
- Последующий вывод из модели и фактическая категория сравниваются для расчета потерь.
- Значение потерь используется для оптимизации весов нейронов в сети.
- После каждых 5000 шагов значение потерь выводится в консоль.

In [127]:
def train(epoch):
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if _%5000==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')

        loss.backward()
        optimizer.step()

In [128]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
2it [00:00, 17.80it/s]

Epoch: 0, Loss:  0.6854485273361206


33it [00:01, 18.55it/s]
3it [00:00, 22.31it/s]

Epoch: 1, Loss:  0.49259236454963684


33it [00:01, 19.41it/s]
3it [00:00, 21.97it/s]

...

33it [00:01, 18.87it/s]
3it [00:00, 21.22it/s]

Epoch: 23, Loss:  0.06059056147933006


33it [00:01, 18.93it/s]
3it [00:00, 21.75it/s]

Epoch: 24, Loss:  0.15749390423297882


33it [00:01, 18.87it/s]


<a id='section06'></a>
### Проверка работоспособности модели

На этапе проверки мы передаем невидимые данные (тестовый набор данных) в модель. Этот шаг определяет, насколько хорошо модель работает с невидимыми данными.

Эти невидимые данные представляют собой 20% файла `train.xlsx`, который был отделен на этапе создания набора данных.
На этапе проверки веса модели не обновляются. Только конечный результат сравнивается с фактическим значением. Это сравнение затем используется для расчета точности модели.

Как сказано выше, чтобы получить оценку производительности наших моделей, мы используем следующие показатели:
- Метрика Хэмминга
- Функция потерь Хэмминга


In [129]:
def validation(testing_loader):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs, fin_targets

In [139]:
outputs, targets = validation(testing_loader)

final_outputs = np.array(outputs) >=0.4

8it [00:00, 41.38it/s]


In [140]:
val_hamming_loss = metrics.hamming_loss(targets, final_outputs)
val_hamming_score = hamming_score(np.array(targets), np.array(final_outputs))

print(f"Hamming Score = {val_hamming_score}")
print(f"Hamming Loss = {val_hamming_loss}")

Hamming Score = 0.499599358974359
Hamming Loss = 0.01644736842105263


<a id='section07'></a>
### Сохранение модели для будущего использования

Это заключительный шаг в процессе точной настройки модели.

Модель и ее словарный запас сохраняются локально. Эти файлы затем используются в будущем для определения новых предоставляемых услуг.

In [ ]:
# Сохраним файлы модели для использования в будущем

output_model_file = './models/pytorch_distilbert_medsi.bin'
output_vocab_file = './models/vocab_distilbert_medsi.bin'

torch.save(model, output_model_file)
tokenizer.save_vocabulary(output_vocab_file)

print('Saved')